# 📘 함수형 프로그래밍 3강: FlatMap과 Maybe/Optional (안전하게 연결하고 펼치기)

---

## 🎯 학습 목표
- `map`과 `flatmap`의 차이를 **사례로 이해**하고 직접 구현한다.
- 중첩(iterable of iterable) 구조를 **평탄화(flatten)** 하는 다양한 방법을 익힌다.
- `Maybe/Optional` 패턴을 파이썬으로 구현하여 **에러/결측(None)** 상황을 안전하게 처리한다.
- 안전한 체이닝(`map → flatmap → map`)으로 **가독성과 테스트 용이성**을 높인다.

## 🧩 핵심 개념
- **Map**: 컨테이너의 각 값을 *함수에 통과*시켜, **형태는 유지**, 내용만 바꾼다.
- **FlatMap**: *함수의 결과가 컨테이너*일 때, 적용 후 **중첩을 한 단계 제거**한다.
- **Maybe/Optional**: 값이 없을 수도 있는 상황을 **명시적 타입/컨테이너**로 다룬다.

## 🔑 키워드
`map`, `flatmap`, `chain.from_iterable`, `리스트 컴프리헨션`, `Optional`, `Maybe`, `Nothing/Just`, `안전한 체이닝`

## ✨ 예시 시나리오
- 게시글 목록 → 각 게시글의 태그 목록 → **모든 태그의 평탄화**
- 문자열 목록 → **정수 파싱**(실패는 건너뛰기) → 짝수만 **제곱**
- 사용자 딕셔너리 → **이메일**(없을 수도 있음) → 소문자 변환


In [1]:

# 🧩 기본 예시: map vs flatmap (리스트 안의 리스트 평탄화)

from itertools import chain
from typing import Iterable, Callable, TypeVar, List

T = TypeVar("T")
U = TypeVar("U")

def map_list(fn: Callable[[T], U], xs: Iterable[T]) -> List[U]:
    """각 원소에 fn을 적용하여 **동일한 컨테이너 모양**을 유지"""
    return [fn(x) for x in xs]

def flatmap_list(fn: Callable[[T], Iterable[U]], xs: Iterable[T]) -> List[U]:
    """fn 결과가 iterable일 때, **중첩을 한 단계 제거**하여 반환"""
    return list(chain.from_iterable(fn(x) for x in xs))

# 예시 데이터: 글 별 태그들
posts = [
    {"id": 1, "tags": ["python", "fp"]},
    {"id": 2, "tags": ["map", "flatmap"]},
    {"id": 3, "tags": []},
]

# map만 쓰면: List[List[str]]
tags_nested = map_list(lambda p: p["tags"], posts)
# flatmap을 쓰면: List[str]
tags_flat   = flatmap_list(lambda p: p["tags"], posts)

print("tags_nested =", tags_nested)
print("tags_flat   =", tags_flat)

assert tags_nested == [["python", "fp"], ["map", "flatmap"], []]
assert tags_flat   == ["python", "fp", "map", "flatmap"]
print("✅ map vs flatmap 기본 예시 통과")


tags_nested = [['python', 'fp'], ['map', 'flatmap'], []]
tags_flat   = ['python', 'fp', 'map', 'flatmap']
✅ map vs flatmap 기본 예시 통과


## 🧩 주요 함수/기능 소개

- **리스트 컴프리헨션**: `ys = [f(x) for x in xs]`
- **`itertools.chain.from_iterable`**: 반복가능한 것들의 **한 단계 평탄화**
- **`flatmap`가 필요한 이유**
  - `map`으로는 중첩이 쌓이기 쉽다: `List[List[T]]`, `Iterator[Iterator[T]]`
  - 중첩을 한 단계 제거해야 다음 계산이 단순해진다.
- **실무 팁**
  - 중첩 구조를 **가능하면 얕게 유지**하고, `flatmap`으로 **즉시 평탄화**하자.
  - Pandas, SQL, Spark 등에서도 동일한 개념이 등장한다(`explode`, `unnest`).

In [2]:

# 🔧 flatmap 다양한 구현

from itertools import chain
from typing import Iterable, Iterator, Callable, TypeVar, List

T = TypeVar("T")
U = TypeVar("U")

def flatmap_list_v1(fn: Callable[[T], Iterable[U]], xs: Iterable[T]) -> List[U]:
    """리스트 컴프리헨션 기반"""
    return [y for x in xs for y in fn(x)]

def flatmap_list_v2(fn: Callable[[T], Iterable[U]], xs: Iterable[T]) -> List[U]:
    """chain.from_iterable 기반"""
    return list(chain.from_iterable(fn(x) for x in xs))

def flatmap_iter(fn: Callable[[T], Iterable[U]], xs: Iterable[T]) -> Iterator[U]:
    """지연(flat) 제너레이터 버전: 대용량에 유리"""
    for x in xs:
        for y in fn(x):
            yield y

# 확인
data = [[1, 2], [], [3]]
assert flatmap_list_v1(lambda z: z, data) == [1, 2, 3]
assert flatmap_list_v2(lambda z: z, data) == [1, 2, 3]
assert list(flatmap_iter(lambda z: z, data)) == [1, 2, 3]
print("✅ flatmap 구현 3종 통과")


✅ flatmap 구현 3종 통과


## 🧩 Maybe/Optional 소개

- **문제**: `None`이 섞인 데이터, 예외가 날 수 있는 처리(파싱/나눗셈/딕셔너리 접근 등)
- **전략**: 값을 직접 꺼내지 않고 **컨테이너**(`Just` or `Nothing`)로 감싸서 **규칙**으로 다룬다.
- **규칙**
  - `map`: 값이 있으면 함수 적용, 없으면 그대로 `Nothing` 유지
  - `flatmap`: 함수가 `Maybe`를 반환할 때, **중첩을 제거**하며 연결
  - `get_or(default)`: 최종적으로 기본값 제공
- **장점**
  - `if x is not None: ... else: ...` 분기를 **체이닝 규약**으로 대체 → 가독성/테스트 용이성 향상


In [7]:

# 🔧 Pythonic Maybe 구현

from dataclasses import dataclass
from typing import Callable, Generic, TypeVar, Union, Iterable

T = TypeVar("T")
U = TypeVar("U")

class Maybe(Generic[T]):
    def map(self, f: Callable[[T], U]) -> "Maybe[U]":
        raise NotImplementedError
    def flatmap(self, f: Callable[[T], "Maybe[U]"]) -> "Maybe[U]":
        raise NotImplementedError
    def get_or(self, default: U | T) -> U | T:
        raise NotImplementedError
    def __repr__(self) -> str:  # 디버깅 편의
        return f"{self.__class__.__name__}()"

@dataclass(frozen=True)
class Just(Maybe[T]):
    value: T
    def map(self, f: Callable[[T], U]) -> "Maybe[U]":
        try:
            return Just(f(self.value))
        except Exception:
            return Nothing()
    def flatmap(self, f: Callable[[T], "Maybe[U]"]) -> "Maybe[U]":
        try:
            return f(self.value)
        except Exception:
            return Nothing()
    def get_or(self, default: U | T) -> U | T:
        return self.value
    def __repr__(self) -> str:
        return f"Just({self.value!r})"

class Nothing(Maybe[T]):
    def map(self, f: Callable[[T], U]) -> "Maybe[U]":
        return self  # 그대로 Nothing
    def flatmap(self, f: Callable[[T], 'Maybe[U]']) -> "Maybe[U]":
        return self  # 그대로 Nothing
    def get_or(self, default: U | T) -> U | T:
        return default
    def __repr__(self) -> str:
        return "Nothing()"

def maybe(x: T | None) -> Maybe[T]:
    """None이면 Nothing, 그 외에는 Just"""
    return Nothing() if x is None else Just(x)

# ✅ 사용 예시 1: 안전한 체이닝
def parse_int(s: str) -> Maybe[int]:
    try:
        return Just(int(s))
    except Exception:
        return Nothing()

def reciprocal(n: int) -> Maybe[float]:
    try:
        return Just(1.0 / n)
    except Exception:
        return Nothing()

result = (
    parse_int("10")        # Just(10)
      .flatmap(reciprocal) # Just(0.1)
      .map(lambda x: round(x, 3))  # Just(0.1)
      .get_or(-1.0)        # 0.1
)
print("예시1 =", result)

# ✅ 사용 예시 2: 딕셔너리 안전 접근
user = {"profile": {"email": "FooBar@EXAMPLE.com"}}

def get(d: dict, k: str) -> Maybe[object]:
    return maybe(d.get(k))

email = (
    get(user, "profile")                # Maybe[dict]
      .flatmap(lambda p: get(p, "email"))  # Maybe[str]
      .map(lambda s: s.lower())         # Maybe[str]
      .get_or("(no email)")
)
print("예시2 =", email)

# ✅ 사용 예시 3: 리스트 파이프라인에서 None 걸러내기
def to_int_or_none(s: str) -> int | None:
    try:
        return int(s)
    except Exception:
        return None

raw = ["1", "x", "2", "y"]
# flatmap으로 None 제거 + 값만 모으기
from itertools import chain
def to_int_maybe_list(s: str):
    n = to_int_or_none(s)
    return [n] if n is not None else []

clean = [v for v in chain.from_iterable(to_int_maybe_list(s) for s in raw)]
print("예시3 =", clean)
assert clean == [1, 2]

print("✅ Maybe/Optional 기본 기능 시연 완료")


예시1 = 0.1
예시2 = foobar@example.com
예시3 = [1, 2]
✅ Maybe/Optional 기본 기능 시연 완료


## 🧪 실습(체험 중심)

이 섹션은 **공통 코드 1개**를 먼저 실행한 뒤, **실습별 코드**를 각각 **독립적으로 실행**해 결과를 확인합니다.  
모든 블록은 콘솔 출력과 `assert`로 자동 검증됩니다. 구현 과제는 아래 **📝 Assignment**에서 진행하세요.

### 실행 순서
1) **실습 공통 코드 실행** → 아래 세 실습에서 공통으로 사용하는 함수와 데이터를 준비합니다.
2) **실습 1 — flatmap 체험**: `[['a'], [], ['b','c']]` → `['a','b','c']`
3) **실습 2 — Maybe 구성 읽기**: `config`에서 `host` 안전 추출 → `'localhost'`
4) **실습 3 — 파이프라인 체험**: 문자열 → 정수 → 짝수 → 제곱 → 기대 `[100, 400]`

> ⚠️ 참고: 실습 코드는 **체험용**이며, 로직은 제공되어 있습니다.


In [1]:

# 🔧 실습 공통 코드 — 먼저 실행하세요

from dataclasses import dataclass
from itertools import chain
from typing import Callable, Generic, Iterable, Iterator, List, TypeVar

T = TypeVar("T")
U = TypeVar("U")

# flatmap & helpers
def flatmap_list(fn: Callable[[T], Iterable[U]], xs: Iterable[T]) -> List[U]:
    '''리스트 컴프리헨션 기반 flatmap'''
    return [y for x in xs for y in fn(x)]

# Maybe/Optional (간단 버전)
class Maybe(Generic[T]):
    def map(self, f: Callable[[T], U]) -> "Maybe[U]": ...
    def flatmap(self, f: Callable[[T], "Maybe[U]"]) -> "Maybe[U]": ...
    def get_or(self, default: U | T) -> U | T: ...

@dataclass(frozen=True)
class Just(Maybe[T]):
    value: T
    def map(self, f: Callable[[T], U]) -> "Maybe[U]":
        try:
            return Just(f(self.value))
        except Exception:
            return Nothing()
    def flatmap(self, f: Callable[[T], "Maybe[U]"]) -> "Maybe[U]":
        try:
            return f(self.value)
        except Exception:
            return Nothing()
    def get_or(self, default: U | T) -> U | T:
        return self.value
    def __repr__(self) -> str:
        return f"Just({self.value!r})"

class Nothing(Maybe[T]):
    def map(self, f: Callable[[T], U]) -> "Maybe[U]":
        return self
    def flatmap(self, f: Callable[[T], "Maybe[U]"]) -> "Maybe[U]":
        return self
    def get_or(self, default: U | T) -> U | T:
        return default
    def __repr__(self) -> str:
        return "Nothing()"

def maybe(x: T | None) -> Maybe[T]:
    return Nothing() if x is None else Just(x)

def get(d: dict, k: str) -> Maybe[object]:
    return maybe(d.get(k))

def parse_db_host(url: str) -> str:
    # 데모용 단순 파서: 'scheme://user@host/db' → host
    at = url.split("@", 1)[1]
    host = at.split("/", 1)[0]
    return host

# 문자열 → 정수 파싱
def to_int_or_none(s: str) -> int | None:
    try:
        return int(s)
    except Exception:
        return None

def to_int_maybe_list(s: str):
    n = to_int_or_none(s)
    return [n] if n is not None else []


In [2]:

# ▶ 실습 1 — flatmap 체험 (그대로 실행)

data = [['a'], [], ['b','c']]
out = flatmap_list(lambda z: z, data)
print("out =", out)
assert out == ['a','b','c']
print("✅ 실습 1 통과")


out = ['a', 'b', 'c']
✅ 실습 1 통과


In [4]:

# ▶ 실습 2 — Maybe 구성 읽기 (그대로 실행)

config = {'db': {'url': 'postgres://user@localhost/mydb'}}

host = (
    get(config, "db")
      .flatmap(lambda d: get(d, "url"))
      .map(str)
      .map(parse_db_host)
      .get_or("(missing)")
)
print("host =", host)
assert host == "localhost"
print("✅ 실습 2 통과")


host = localhost
✅ 실습 2 통과


In [7]:

# ▶ 실습 3 — 파이프라인 체험 (그대로 실행)

raw = ['10','-','20','x','7']

# (a) 정수 파싱 실패 건너뛰기 → flatmap으로 None 제거
nums = [v for v in chain.from_iterable(to_int_maybe_list(s) for s in raw)]
# (b) 짝수만 선택
evens = [n for n in nums if n % 2 == 0]
# (c) 제곱
squared = [n*n for n in evens]

print("nums    =", nums)
print("evens   =", evens)
print("squared =", squared)

assert nums == [10, 20, 7]
assert evens == [10, 20]
assert squared == [100, 400]
print("✅ 실습 3 통과")


nums    = [10, 20, 7]
evens   = [10, 20]
squared = [100, 400]
✅ 실습 3 통과


## ⚡ 정리
- `map`: 형태 유지, 내용만 변환
- `flatmap`: **중첩 컨테이너 한 단계 제거**하며 연결
- `Maybe/Optional`: 결측/예외를 **규약으로 모델링**해서 if-else 분기를 체이닝 규칙으로 대체

## 📝 Assignment
1) **`lazy_flatmap`**을 제너레이터로 구현하고, 100만 행에서도 **메모리 초과 없이** 동작하는지 확인하세요.
2) **`safe_get_in(d, path)`**를 `Maybe`로 구현하세요. 예: `safe_get_in(user, ["profile","email"])` → 값 또는 `Nothing`.
3) **파이프라인 강화**: 문자열 리스트에서 정수 파싱 → 짝수 → 제곱 → 100 이상만 출력. `flatmap`과 컴프리헨션을 적절히 조합하세요.

## 규칙
- 모든 예제는 **즉시 실행 가능**하게 유지합니다.
- I/O(파일/네트워크) 등 사이드 이펙트는 **강의 범위 밖**으로 격리하고, 여기서는 **순수 함수**만 다룹니다.
